In [ ]:
from sumo.wrapper import SumoClient
from fmu.sumo.explorer import Explorer

sumoclient = SumoClient(env="dev")

explorer = Explorer(env="dev")

<h1>Find all cases for specific asset with status=status</h1>

In [ ]:
asset = "Johan Sverdrup"
status = "keep"

In [ ]:
# WRAPPER GET: find all cases for specific asset with status=status
searchparams = {
    "$query": f"access.asset.name:\"{asset}\" AND _sumo.status:{status}"
}
cases = sumoclient.get("/searchroot", params=searchparams).json()["hits"]["hits"]
print(len(cases))


In [ ]:
# WRAPPER POST: find all cases for specific asset with status=status
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "term": {
                        "access.asset.name.keyword": {
                            "value": asset
                        }
                    }
                },
                {
                    "term": {
                        "_sumo.status.keyword": {
                            "value": status
                        }
                    }
                }
            ]
        }
    }
}
cases = sumoclient.post("/search", json=query).json()["hits"]["hits"]
print(len(cases))

In [ ]:
# EXPLORER: find all cases for specific asset with status=status
explorer_cases = explorer.cases.filter(
    status=status,
    asset=asset
)
print(len(explorer_cases))
print(explorer_cases[0]["fmu"]["case"]["uuid"])

In [ ]:
# WRAPPER POST: find all cases for MULTIPLE assets with status=status
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "terms": {
                        "access.asset.name.keyword": [
                            "Johan Sverdrup", "Drogon"
                        ]
                    }
                },
                {
                    "term": {
                        "_sumo.status.keyword": {
                            "value": status
                        }
                    }
                }
            ]
        }
    },
    "size": 100
}
cases = sumoclient.post("/search", json=query).json()["hits"]["hits"]
print(len(cases))

<h1>Find all surfaces of a given case</h1>

In [ ]:
caseuuid = "ac1a2df9-961e-4a54-82a6-7da7283a2942" # "c8047a47-0652-483e-860f-fd62dbaf16a7"

In [ ]:
# EXPLORER: find all surfaces for a case
pit_explorer = Explorer(env="dev", keep_alive="5m")
mycase = pit_explorer.get_case_by_uuid(caseuuid)

surfaces = mycase.surfaces

print(len(set(surfaces)))

In [ ]:
# Without keep_alive
no_pit_explorer = Explorer(env="dev")
mycase = no_pit_explorer.get_case_by_uuid(caseuuid)

surfaces = mycase.surfaces

print(len(set(surfaces)))

In [ ]:
# WRAPPER with pit:
pit = sumoclient.post("/pit", params={"keep_alive": "5m"}).json()["id"]

surfaces = []
search_after = None
while True:
    params = {
        "$query": f"class:surface AND _sumo.parent_object:{caseuuid}",
        "$sort": "_doc:asc",
        "$size": 100,
        "$select": "false",
        "$pit": pit,
        "$pit_keepalive": "5m" 
        }
    if search_after is not None:
        params["$search_after"] = search_after
    res = sumoclient.get(f"/search", params=params).json()
    surfs = res["hits"]["hits"]
    if len(surfs) == 0:
        break

    pit = res["pit_id"]
    search_after = str(surfs[-1]["sort"])
    surfaces += [s["_id"] for s in surfs]

print(len(set(surfaces)))

In [ ]:
# WRAPPER WITHOUT pit: !! Not guaranteed to get everything
surfaces = []
search_after = None
while True:
    params = {
        "$query": f"class:surface AND _sumo.parent_object:{caseuuid}",
        "$sort": "_doc:asc",
        "$size": 50,
        "$select": "false",
        }
    if search_after is not None:
        params["$search_after"] = search_after
    res = sumoclient.get(f"/search", params=params).json()
    surfs = res["hits"]["hits"]
    if len(surfs) == 0:
        break

    search_after = str(surfs[-1]["sort"])
    surfaces += [s["_id"] for s in surfs]

print(len(set(surfaces)))

<h1>Nested aggregates</h1>

In [ ]:
# WRAPPER: nested aggregates
query = {
  "query": {
    "term": {
      "_sumo.parent_object.keyword": {
        "value": caseuuid
      }
    }
  },
  "size": 0,
  "track_total_hits": "true",
  "aggs": {
    "class": {
      "terms": {
        "field": "class.keyword",
        "size": 20
      },
      "aggs": {
        "name": {
          "terms": {
            "field": "data.name.keyword",
            "size": 100
          },
          "aggs": {
            "tagname": {
              "terms": {
                "field": "data.tagname.keyword",
                "size": 100
              }
            }
          }
        },
        "blobstats": {
          "stats": {
            "field": "_sumo.blob_size"
          }
        }
      }
    }
  }
}
res = sumoclient.post("/search", json=query).json()

import json
print(json.dumps(res, indent=4))